In [19]:
import queue
import sqlite3
import datetime
import tkinter as tk
from tkinter import messagebox, scrolledtext
import logging
from concurrent.futures import ThreadPoolExecutor
import time

# ---- Logging ----/
logging.basicConfig(filename="lifecoach_errors.log", level=logging.ERROR,
                    format="%(asctime)s %(levelname)s %(message)s")

# ---- LLM Setup ----
AGENT_AVAILABLE = False
llm = None

try:
    from langchain_community.llms import Ollama
    llm = Ollama(model="llama2")
    AGENT_AVAILABLE = True
except Exception as e:
    logging.error("Ollama not available or failed to initialize: %s", e)
    AGENT_AVAILABLE = False

# ---- Database Setup ----
DB_PATH = "lifecoach.db"

def init_db():
    conn = sqlite3.connect(DB_PATH, check_same_thread=False)
    cur = conn.cursor()
    cur.execute("""
    CREATE TABLE IF NOT EXISTS moods (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        user_input TEXT,
        llm_response TEXT,
        created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
    )
    """)
    conn.commit()
    return conn

_main_conn = init_db()
_main_conn.row_factory = sqlite3.Row

def get_conn():
    conn = sqlite3.connect(DB_PATH, check_same_thread=False)
    return conn

# ---- Thread & Queue ----
result_q = queue.Queue(maxsize=100)
executor = ThreadPoolExecutor(max_workers=5)

# ---- LLM call wrapper ----
def safe_agent_run(prompt):
    if AGENT_AVAILABLE and llm:
        try:
            resp = llm.invoke(prompt)
            if resp:
                return resp
        except Exception as e:
            logging.error("Direct LLM call failed: %s", e)
    return "Sentiment: NEUTRAL\nSuggestion: Sorry, the LLM is currently unavailable."

# ---- Worker Thread ----
def worker_thread(user_input, context_text):
    try:
        prompt = (
            "You are an empathetic AI Life Coach.\n\n"
            "Instructions:\n"
            "1. Detect the sentiment accurately: POSITIVE, NEGATIVE, or NEUTRAL.\n"
            "2. Provide a concise, actionable suggestion (1 sentence maximum).\n"
            "3. Do NOT include greetings or filler phrases.\n"
            "4. Keep your response short (max 2 lines):\n\n"
            "Output EXACTLY as:\n"
            "Sentiment: <POSITIVE/NEGATIVE/NEUTRAL>\n"
            "Suggestion: <Your advice>\n\n"
        )
        if context_text:
            prompt += f"Previous conversation (most recent first):\n{context_text}\n\n"
        prompt += f"User says: \"{user_input}\"\n"

        llm_response = safe_agent_run(prompt)

        try:
            conn = get_conn()
            cur = conn.cursor()
            cur.execute(
                "INSERT INTO moods (user_input, llm_response, created_at) VALUES (?, ?, ?)",
                (user_input, llm_response, datetime.datetime.utcnow())
            )
            conn.commit()
            conn.close()
        except Exception as e:
            logging.error("DB insert error in worker_thread: %s", e)

        try:
            result_q.put(llm_response, block=False)
        except queue.Full:
            logging.error("result_q is full; dropped LLM response")
    except Exception as e:
        logging.error("worker_thread error: %s", e)
        try:
            result_q.put("An error occurred while generating the suggestion.")
        except queue.Full:
            pass

# ---- GUI ----
class LifeCoachGUI:
    USER_COLOR = "#DFF7DF"

    def __init__(self, root):
        self.root = root
        root.title("🌟 AI Life Coach (LLM-based) 🌟")
        self.last_submit_time = 0.0
        self.placeholder = "How are you feeling today?"
        self.typing_tag_name = "typing"
        self._build_ui()
        self._poll_queue()
        self._display_initial_message()
        root.protocol("WM_DELETE_WINDOW", self.on_close)

    def _build_ui(self):
        self.chat = scrolledtext.ScrolledText(self.root, height=18, state=tk.DISABLED, wrap=tk.WORD, font=("Arial", 11))
        self.chat.pack(fill=tk.BOTH, padx=8, pady=(6,0), expand=True)

        entry_frame = tk.Frame(self.root)
        entry_frame.pack(fill=tk.X, padx=8, pady=6)

        self.mood_entry = tk.Text(entry_frame, height=2, font=("Arial", 12))
        self.mood_entry.pack(side=tk.LEFT, fill=tk.X, expand=True, padx=(0,6))
        self.mood_entry.insert("1.0", self.placeholder)
        self.mood_entry.bind("<FocusIn>", self._on_entry_focus_in)
        self.mood_entry.bind("<FocusOut>", self._on_entry_focus_out)
        self.mood_entry.bind("<Return>", self._on_enter_pressed)

        submit_btn = tk.Button(entry_frame, text="Submit", command=self.on_submit, bg="#4CAF50", fg="white")
        submit_btn.pack(side=tk.LEFT)

        bottom_frame = tk.Frame(self.root)
        bottom_frame.pack(fill=tk.X, padx=8, pady=(0,4))

        clear_btn = tk.Button(bottom_frame, text="Clear chat", command=self.clear_chat, bg="#A9A9A9", fg="white")
        clear_btn.pack(side=tk.LEFT)

        self.status_var = tk.StringVar(value="Ready")
        status_label = tk.Label(self.root, textvariable=self.status_var, anchor="w")
        status_label.pack(fill=tk.X, padx=8, pady=(0,6))

    def _on_entry_focus_in(self, event):
        current = self.mood_entry.get("1.0", tk.END).strip()
        if current == self.placeholder:
            self.mood_entry.delete("1.0", tk.END)

    def _on_entry_focus_out(self, event):
        current = self.mood_entry.get("1.0", tk.END).strip()
        if not current:
            self.mood_entry.delete("1.0", tk.END)
            self.mood_entry.insert("1.0", self.placeholder)

    def _on_enter_pressed(self, event):
        if event.state & 0x1:
            self.mood_entry.insert(tk.INSERT, "\n")
            return "break"
        else:
            self.on_submit()
            return "break"

    def _log(self, sender, text):
        ts = datetime.datetime.now().strftime("%H:%M:%S")
        self.chat.config(state=tk.NORMAL)
        self.chat.insert(tk.END, f"[{ts}] {sender}: ", ("sender",))
        self.chat.tag_configure("sender", font=("Arial", 10, "bold"))
        self.chat.insert(tk.END, f"{text}\n\n", ("msg",))
        self.chat.tag_configure("msg", foreground="black")
        self.chat.see(tk.END)
        self.chat.config(state=tk.DISABLED)

    def _display_initial_message(self):
        self._log("Coach", "Hello! I'm your AI Life Coach. Tell me how you're feeling, and I'll provide guidance.")

    def clear_chat(self):
        self.chat.config(state=tk.NORMAL)
        self.chat.delete("1.0", tk.END)
        self.chat.config(state=tk.DISABLED)
        self._display_initial_message()

    def on_submit(self):
        now = time.time()
        if now - self.last_submit_time < 0.15:
            return
        self.last_submit_time = now

        user_input = self.mood_entry.get("1.0", tk.END).strip()
        if not user_input or user_input == self.placeholder:
            messagebox.showwarning("Input error", "Please enter a mood or how you feel.")
            return

        self._log("You", user_input)
        self._show_typing_indicator()
        context_text = self._fetch_recent_context(limit=3)

        try:
            executor.submit(worker_thread, user_input, context_text)
        except Exception as e:
            logging.error("Failed to submit worker_thread: %s", e)
            self._log("Coach", "⚠️ Internal error: couldn't process your request.")
            self._hide_typing_indicator()

        self.mood_entry.delete("1.0", tk.END)

    def _show_typing_indicator(self):
        self.chat.config(state=tk.NORMAL)
        ts = datetime.datetime.now().strftime("%H:%M:%S")
        self.chat.insert(tk.END, f"[{ts}] Coach: typing...\n\n", (self.typing_tag_name,))
        self.chat.tag_configure(self.typing_tag_name, foreground="gray", font=("Arial", 10, "italic"))
        self.chat.see(tk.END)
        self.chat.config(state=tk.DISABLED)
        self.status_var.set("Coach is typing...")

    def _hide_typing_indicator(self):
        self.chat.config(state=tk.NORMAL)
        ranges = self.chat.tag_ranges(self.typing_tag_name)
        if ranges:
            for i in range(0, len(ranges), 2):
                start = ranges[i]
                end = ranges[i+1]
                try:
                    self.chat.delete(start, end)
                except Exception:
                    pass
        self.chat.config(state=tk.DISABLED)
        self.status_var.set("Ready")

    def _poll_queue(self):
        try:
            while True:
                llm_response = result_q.get_nowait()
                self._hide_typing_indicator()
                self._log("Coach", llm_response)
        except queue.Empty:
            pass
        self.root.after(150, self._poll_queue)

    def _fetch_recent_context(self, limit=3):
        try:
            cur = _main_conn.cursor()
            cur.execute("SELECT user_input, llm_response FROM moods ORDER BY id DESC LIMIT ?", (limit,))
            rows = cur.fetchall()
            if not rows:
                return ""
            parts = []
            for r in rows:
                ui = r["user_input"] if isinstance(r, sqlite3.Row) else r[0]
                lr = r["llm_response"] if isinstance(r, sqlite3.Row) else r[1]
                parts.append(f"You: {ui}\nCoach: {lr}")
            return "\n\n".join(parts)
        except Exception as e:
            logging.error("fetch_recent_context error: %s", e)
            return ""

    def on_close(self):
        try:
            self.status_var.set("Shutting down...")
            try:
                executor.shutdown(wait=False)
            except Exception as e:
                logging.error("Error shutting down executor: %s", e)
            try:
                _main_conn.close()
            except Exception:
                pass
            self.root.after(200, self.root.destroy)
        except Exception as e:
            logging.error("on_close error: %s", e)
            try:
                self.root.destroy()
            except Exception:
                pass

if __name__ == "__main__":
    root = tk.Tk()
    app = LifeCoachGUI(root)
    root.mainloop()


C:\Users\Tayyab\AppData\Local\Temp\ipykernel_11716\2267685985.py:90: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  (user_input, llm_response, datetime.datetime.utcnow())
C:\Users\Tayyab\AppData\Local\Temp\ipykernel_11716\2267685985.py:90: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  (user_input, llm_response, datetime.datetime.utcnow())
C:\Users\Tayyab\AppData\Local\Temp\ipykernel_11716\2267685985.py:90: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  (user_input, llm_response, datetime.datetime.utcnow())
C:\Users\Tayy